# 1. Importar tablas

In [1]:
timeorigin: .z.t

In [2]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [3]:
/ 1.2. Importar tabla de las bases de contribución
contributionBasis:get `:data/contributionBasis;

/show employees

In [4]:
/ 1.3. Importar tabla de tipos
rateTypes:("DDFSsS";enlist csv) 0: `$":data/rateTypes.csv";

/show rateTypes

In [5]:
/ 1.3. Importar tabla de escala irpf
irpfScale:("DDFF";enlist csv)0:`$":data/irpfScale.csv";

irpfScale: ungroup select range, rate, acc:sums prev[rate]*deltas range
                   by startDate, endDate 
                   from irpfScale;

/show irpfScale

# 2. Variables globales

In [6]:
cnaes:string asc distinct rateTypes`cnae;                                 /all the cnae's strings

# 3. Funciones


## 3.1. Cálculo de las aportaciones

In [7]:
gacnae:{`$cnaes last where cnaes in (neg til count string x)_\:string x}; /(g)et (a)ctual (cnae)

gRate:{[mo;conti;contr;cn]                                                /(g)et (rate) type
    0^first exec rate%100 
            from rateTypes 
            where contingency=conti, 
                  contributor=contr, 
                  mo within (startDate;endDate),
                  (cnae=cn) | cnae=`0};

## 3.2. Cálculo de las retenciones

In [8]:
calirpf:{[n;ye]                                                           /(c)alculate (irpf)
    first exec acc+rate*n-range 
          from (select from irpfScale where ye within (`year$startDate;`year$endDate))
          where i=range bin n};

gIrpfRate:{[mo;ni;contr]                                                  /(g)et (irpf) (rate) type
    if[contr=`E;:0];
    r:first exec irpfRate 
           from irpfBasis 
           where nif=ni,year=`year$mo;
    $[r=0;0;r|0.02]};

## 3.3. Funciones para monthlyContributions

In [9]:
gconty:{[bt;ws]                                                           /(g)en (con)tribution (ty)pe
    d: `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase!
    ((`cc;(`at;$[ws=`temp;`ded;`dei];`fp;`fgs);`hef;`he;`irpf;`mei));
    d bt};

cmonthly_contributions:{[nif;mon;contributor;conType;cnae]              /(c)reate monthlyContributions
        $[conType=`irpf;
          :gIrpfRate[mon;nif;contributor];
          :gRate[mon;conType;contributor;cnae]];};

# 4. Generación de las tablas

In [10]:
/---------- monthlyContributionsla auxiliar irpfBasis ----------
irpfBasis: () xkey select aIrpfBase: sum amount, mpf:5500f                             /new monthlyContributionsle with: (a)ctual (irpfBase)  & "(m)ínimo (p)ersonal y (f)amiliar"
                   by year:`year$month, nif 
                   from contributionBasis
                   where baseType=`irpfBase;

update retfee: 0 | calirpf'[aIrpfBase;year] - calirpf'[mpf;year]                      /add a column for (ret)ention (fee)
    from `irpfBasis;

irpfBasis: irpfBasis lj select contractType by nif from employees;                    /add contract type

update irpfRate: {$[first[x]=`temp;0.02;0f] | 0.01 xbar 0.005 + y%z}'[contractType;retfee;aIrpfBase]  /add a column for the irpf Rate
    from `irpfBasis;

delete aIrpfBase, mpf,retfee,contractType from `irpfBasis;
/---------- monthlyContributionsla auxiliar irpfBasis ----------

In [11]:
monthlyContributions: update contributionType: gconty'[baseType;workingSchedule], cnae: gacnae'[cnae]
                      from contributionBasis lj 
                           `nif xkey select workingSchedule, cnae, nif from employees;

In [12]:
monthlyContributions: raze {n: 2*count x[`contributionType];
     flip `nif`month`contributor`contributionType`amount`cnae!(
        n#x[`nif];
        n#x[`month];
        ((n div 2)#`E),(n div 2)#`T;
        (x[`contributionType]),x[`contributionType];
        n#x[`amount];
        n#x[`cnae])} each monthlyContributions;

In [13]:
monthlyContributions: update amount: amount*cmonthly_contributions'[nif;month;contributor;contributionType;cnae]
                      from monthlyContributions;

In [14]:
monthlyContributions: select nif, month, contributor, contributionType, amount 
                      from `monthlyContributions 
                      where amount<>0;

In [15]:
timeend: .z.t

Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [16]:
show 0^exec `cc`at`fp`fgs`hef`he`irpf`mei#(contributionType!amount)
    by nif,month,contributor
    from monthlyContributions;

nif       month   contributor| cc       at       fp       fgs      hef he    ..
-----------------------------| ----------------------------------------------..
00193031S 2023.01 E          | 453.5189 63.41578 11.53014 3.843381 0   0     ..
00193031S 2023.01 T          | 90.31944 0        1.92169  0        0   0     ..
00193031S 2023.02 E          | 595.7517 83.30426 15.14623 5.048743 0   0     ..
00193031S 2023.02 T          | 118.6455 0        2.524371 0        0   0     ..
00193031S 2023.03 E          | 585.6787 81.89575 14.89014 4.963379 0   0     ..
00193031S 2023.03 T          | 116.6394 0        2.481689 0        0   0     ..
00193031S 2023.04 E          | 599.9425 87.48556 15.90647 5.302155 0   25.711..
00193031S 2023.04 T          | 119.4801 0        2.651078 0        0   5.1205..
00193031S 2023.05 E          | 598.8376 86.42727 15.71405 5.238016 0   19.248..
00193031S 2023.05 T          | 119.26   0        2.619008 0        0   3.8333..
00193031S 2023.06 E          | 592.1262 

# 5. Guardar datos en csv

In [17]:
`:data/irpfBasis set irpfBasis;
`:data/monthlyContributions set monthlyContributions;

In [18]:
"Number of contribution: ", string count monthlyContributions

"Total time spent: ",string timeend-timeorigin

"Number of contribution: 642437"


"Total time spent: 00:00:04.794"
